In [ ]:
from pyspark.sql import SparkSession

#créer une session dans le master
spark = SparkSession.builder \
    .master("spark://172.20.53.96:7077") \
    .appName("WDC-CSET+domains+sample") \
    .config("spark.executor.memory","28g") \
    .config("spark.driver.memory","28g") \
    .getOrCreate()
#spark = SparkSession.builder.master("local").appName("WDC-CSET+domains+sample").getOrCreate()
spark.conf.set("spark.worker.cleanup.enabled",True)
spark.conf.set("spark.worker.cleanup.interval",1800)
spark.conf.set("spark.worker.cleanup.appDataTtl",3600)
spark.conf.set("spark.sql.shuffle.partitions",1000)

#fichiers de config qui permettent de se connecter au serveur de stockage s3 qui contient les fichiers de DataCommons
endpoint_url = 'https://s3.os-bird.glicid.fr/'
aws_access_key_id = '***REMOVED***'
aws_secret_access_key = '***REMOVED***'
hadoopConf = spark._jsc.hadoopConfiguration()
hadoopConf.set('fs.s3a.access.key', aws_access_key_id)
hadoopConf.set('fs.s3a.secret.key', aws_secret_access_key)
hadoopConf.set('fs.s3a.endpoint', endpoint_url)
hadoopConf.set('fs.s3a.path.style.access', 'true')
hadoopConf.set('fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem')

hadoopConf.set('spark.worker.cleanup.enabled', 'true')
hadoopConf.set('fs.s3a.committer.name', 'magic')

In [ ]:
from pyspark.sql.functions import col, count
from pyspark.sql import functions as f

from pyspark.sql import Row
import re
from urllib.parse import urlparse

import subprocess


line0 = '_:nb53a244 <http://schema.org/value1> "8,840 ft<> / 2,694 m<>" <https://youtube.com>   .'
line1 = '_:nb53a244 <http://schema.org/value2> "8,840 ft<> / 2,694 m<>" <https://youtube.com>   .'
line2 = '_:different <http://schema.org/value3> "zfgrredzfzef" <https://youtube.com>   .'
line3 = '_:nb53a244 <http://schema.org/value4> "dqqsd" <https://different.com>   .'

line4 = '<https://google.com> <http://schema.org/value4> "8,840 ft<> / 2,694 m<>" <https://youtube.com>   .'
line5 = '<https://google.com> <http://schema.org/value4> "8,840 ft<> / 2,694 m<>" <https://youtube.co.uk>   .'
line6 = '<https://different.com> <http://schema.org/value6> "fsqfqsfdqs" <https://youtube.com>   .'
line7 = '<https://google.com> <http://schema.org/type> "Recipe" <https://different.com>   .'

line8 = '<https://google.com> <http://schema.org/type> "Recipe"'
line9 = '<https://google.com> <http://schema.org/type>'
line10 = '<https://google.com>'
line11 = ''

# pip install tldextract pas internet
from pyspark.sql.functions import udf
import tldextract

@udf('string')
def extract_tld(url):
    tld = tldextract.extract(url)
    if tld.registered_domain:
        return tld.registered_domain
    else:
        # in case the URL is an IP
        return tld.domain

spark.udf.register("extract_tld", extract_tld)

def extract_tld_func(url):
    tld = tldextract.extract(url)
    if tld.registered_domain:
        return tld.registered_domain
    else:
        # in case the URL is an IP
        return tld.domain

quad_motif = re.compile(r'([^\s]+)\s([^\s]+)\s(.+)\s([^\s]+)\s+\.')

def parseQ(l, parts):
    result = quad_motif.match(l)
    if result:
        sub = result.group(1).strip()
        
        pred = result.group(2).strip()
        pred = re.sub(r'([Hh][Tt][Tt][Pp][Ss]?://)?([Ww]{3}\.)?', '', pred)
        if pred == "<w3.org/1999/02/22-rdf-syntax-ns#type>":
            
            pred = "isa:" + re.sub(r'([Hh][Tt][Tt][Pp][Ss]?://)?([Ww]{3}\.)?', '', result.group(3).strip())

        if sub.startswith("_:"):
            sub += result.group(4).strip()

        #hashstring = extracted.domain + "." + extracted.suffix
        hashstring = extract_tld_func(result.group(4).strip().strip("<>"))
        return Row(subject=sub, predicate=pred, hashdom=hash(hashstring) % parts)
    else:
        # Handle non-matching lines
        print(f"parsing error : {l}")
        return None

print(parseQ(line0, 100))
print(parseQ(line1, 100))
print(parseQ(line2, 100))
print(parseQ(line3, 100))
print(parseQ(line4, 100))
print(parseQ(line5, 100))
print(parseQ(line6, 100))
print(parseQ(line7, 100))
print(parseQ(line8, 100))
print(parseQ(line9, 100))
print(parseQ(line10, 100))
print(parseQ(line11, 100))

In [ ]:
from pyspark.sql.types import ArrayType, StringType
import random 

from pyspark.sql.functions import col
def partCS(hashdom_val, hashdom_max, output=None):
    print(f"Working on part {hashdom_val+1}/{hashdom_max}.")
    file_name = f"{output}/hashdom{hashdom_val+1}-{hashdom_max}"
    
    #CONCAT_WS(' ', sample_tab(COLLECT_LIST(regexp_replace(subject, '^[^<>]*<(.+)>[^<>]*$', '$1')))) as urls
    #COUNT(distinct extract_tld(regexp_replace(subject, '^[^<>]*<(.+)>[^<>]*$', '$1'))) as distinct_domains_count,
    result = spark.sql(f"""
        SELECT
            pset,
            COUNT(subject) as count,
            CONCAT_WS(' ',slice(shuffle(COLLECT_LIST(regexp_replace(subject, '^[^<>]*<(.+)>[^<>]*$', '$1'))), 1,100)) as sample,
            COUNT(distinct extract_tld(regexp_replace(subject, '^[^<>]*<(.+)>[^<>]*$', '$1'))) as distinct_domains_count
        FROM (
            SELECT
                subject,
                CONCAT_WS(' ', SORT_ARRAY(COLLECT_SET(predicate))) as pset
            FROM
                Super
            WHERE
                hashdom={hashdom_val}
            GROUP BY
                subject
        )
        GROUP BY
            pset
    """).cache()
    
    result.show(truncate=0)

    if(output is not None):
        print("Saving")
        result.write.option("header",True) \
            .option("compression","gzip") \
            .mode("overwrite") \
            .parquet(file_name)
    
    # clear variables from memory
    result.unpersist()

    print(f"part {hashdom_val+1}/{hashdom_max} finished");
    
    del result

def completeCS(input, parts, output=None):
    lines = spark.sparkContext.textFile(input)    
    if lines.isEmpty():
        print(f"error: {input} folder empty")
    
    parsedLines = lines.map(lambda l: parseQ(l, parts)).filter(lambda x: x is not None).toDF()
    parsedLines.createOrReplaceTempView("Super")
    
    # Assurez-vous que la fonction check_hashdom_existence_with_hadoop_command est correctement définie comme montré précédemment
    for i in range(parts):
        if not check_hashdom_existence_with_hadoop_command(i):
            # Appelle partCS seulement si le fichier _SUCCESS n'existe pas pour le hashdom actuel
            partCS(i, parts, output)
        else:
            print(f"Skipping part {i+1}/{parts} as it already exists with a _SUCCESS file.")
    
    print("Finished")


    
import subprocess

def check_hashdom_existence_with_hadoop_command(hashdom_val):
    """Vérifie si le fichier _SUCCESS existe pour un hashdom donné en utilisant une commande hadoop fs avec chemin complet."""
    success_file_path = f"s3a://ter-2024/ddomain-240209/hashdom{hashdom_val+1}-20/_SUCCESS"
    hadoop_executable_path = "/webdatacommons_data/soft/hadoop-3.3.6/bin/hadoop"  # Chemin complet vers hadoop
    command = [hadoop_executable_path, 'fs', '-test', '-e', success_file_path]
    
    try:
        result = subprocess.run(command, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        # Si la commande réussit, le fichier existe
        return True
    except subprocess.CalledProcessError:
        # Si la commande échoue, le fichier n'existe pas
        return False



In [ ]:
# input : test or wdc or wdc-2023
# no output = no save
#  s3a://wdc/dpef.html-embedded-jsonld.nq-06239.gz
#completeCS("s3a://wdc/dpef.html-embedded-jsonld.nq-06239.gz", 20, "s3a://ter-2024/ddomain-240207")
completeCS("s3a://wdc/**", 20, "s3a://ter-2024/ddomain-240209")